# Assignment 7

## Submit as an HTML file

### Print your name below

In [1]:
print("Joy Qu")

Joy Qu


### Import the "pandas" "numpy" and "statsmodels.formula.api" libraries

In [2]:
# Import the pandas library
import pandas as pd

# Import the numpy library
import numpy as np

# Import the statsmodels library
import statsmodels.formula.api as smf

#### In the code chunk below read the CSV file named `results.csv` in the `data` <br> folder and print the first 5 rows of the dataset. Browse the dataset.

In [ ]:
# Read the results dataset
results = pd.read_csv('data/results.csv')

# Print the first 5 rows of the dataset
print(results.head())

   resultId  raceId  driverId  ...  fastestLapTime fastestLapSpeed  statusId
0         1      18         1  ...        1:27.452         218.300         1
1         2      18         2  ...        1:27.739         217.586         1
2         3      18         3  ...        1:28.090         216.719         1
3         4      18         4  ...        1:28.603         215.464         1
4         5      18         5  ...        1:27.418         218.385         1

[5 rows x 18 columns]


### (a)  Check Column Types and Data Cleaning

- Use the function .dtypes to get the column types
- Identify which columns have data types that might need conversion
- The 'milliseconds' column contains string values that should be numeric. Create a new column called 'race_time_ms' that:
    - Converts the column to a numeric data type
    - Replaces any non-numeric values with NaN

In [ ]:
# Write your answer here
# Print the column types of 'results'
print(results.dtypes)

# Convert the 'milliseconds' column to numeric
# Replace non-numeric values with NaN
results['race_time_ms'] = pd.to_numeric(results['milliseconds'], errors = 'coerce')

# Display the cleaned dataset
display(results)

resultId             int64
raceId               int64
driverId             int64
constructorId        int64
number              object
grid                 int64
position            object
positionText        object
positionOrder        int64
points             float64
laps                 int64
time                object
milliseconds        object
fastestLap          object
rank                object
fastestLapTime      object
fastestLapSpeed     object
statusId             int64
race_time_ms       float64
dtype: object


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,race_time_ms
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1,5690616.0
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1,5696094.0
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1,5698779.0
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1,5707797.0
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1,5708630.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,25841,1096,854,210,47,12,16,16,16,0.0,57,\N,\N,39,12,1:29.833,211.632,11,NaN
25836,25842,1096,825,210,20,16,17,17,17,0.0,57,\N,\N,40,20,1:31.158,208.556,11,NaN
25837,25843,1096,1,131,44,5,18,18,18,0.0,55,\N,\N,42,11,1:29.788,211.738,9,NaN
25838,25844,1096,849,3,6,20,19,19,19,0.0,55,\N,\N,45,14,1:30.309,210.517,130,NaN


### (b) Create Categorical Variables

- Create a new column called 'finish_category' that categorizes the race finish positions as follows:
    - Positions 1-3: 'Podium'
    - Positions 4-10: 'Points'
    - Positions 11-20: 'Midfield'
    - Positions >20: 'Backmarker'

Hint: Use the pd.cut() function

In [ ]:
# Write your answer here
# Define the bins and labels
bins_x = [1, 3, 10, 20, float("inf")]
labels_x = ['Podium', 'Points', 'Midfield', 'Backmarker']

# Create a new column 'finish_category'
# Categorize the race finish positions based on the bins
results['finish_category'] = pd.cut(results['positionOrder'],
                                    bins = bins_x,
                                    right = True,
                                    labels = labels_x)

### (c) Calculate Race Duration
- For rows where 'milliseconds' is available, create a new column <br>
'race_duration_minutes' that converts milliseconds to minutes by dividing <br>
by (1000*60).
- Display the average race duration by 'constructorId' for the top 5 <br>
constructors with the shortest average race times

In [ ]:
# Write your answer here
# Replace \N values with float NaN values
results["milliseconds"] = results["milliseconds"].replace("\\N", float("nan"))

# Calculate the race duration in minutes
results['race_duration_minutes'] = results["milliseconds"].astype(float) / (1000 * 60)

# Group 'results' by 'constructorId'
results_agg = (results.groupby("constructorId")
               
# Compute the average race duration
                      .agg(mean_racetime = ('race_duration_minutes','mean'))

# Sort the results by 'mean_racetime'
                      .sort_values(by = 'mean_racetime', ascending = True))

# Display the top 5 constructors
print(results_agg.head())

               mean_racetime
constructorId               
35                 76.710777
29                 77.604125
41                 87.046767
16                 89.428828
53                 89.658852


### (d) Driver Performance Analysis

- Calculate the following statistics for each driver, grouped by 'driverId':
    - Average finishing position
    - Total points
    - Number of races completed
    - Best finishing position

- Sort the results by total points in descending order
- Display the top 10 drivers based on total points

In [41]:
# Write your answer here
# Group 'results' by 'driverId'
drivers_agg = (results.groupby('driverId')
               
# Compute the average finishing position
                      .agg(mean_position = ('positionOrder','mean'),

# Compute the total points
                           sum_points =   ('points','sum'),

# Compute the number of completed races
                           count_races =  ('raceId', len),

# Compute the best finishing position
                           min_position =  ('positionOrder','min'))

# Sort the results by 'sum_points' 
                        .sort_values(by = 'sum_points', ascending = False))

# Print the top 10 drivers
print(drivers_agg.head(10))

          mean_position  sum_points  count_races  min_position
driverId                                                      
1              4.787097      4396.5          310             1
20             7.093333      3098.0          300             1
4              8.494413      2061.0          358             1
830            6.533742      1983.5          163             1
8              8.491477      1873.0          352             1
822            7.601990      1778.0          201             1
3              8.252427      1594.5          206             1
30             6.879870      1566.0          308             1
817            9.883621      1307.0          232             1
18             9.695793      1235.0          309             1


### (e) Linear Regression
Create a linear regression model that predicts 'points' based on 'grid' (starting position) and 'laps' completed <br>
Use the following steps:

- Clean the data to remove any non-numeric values and missing values
- Create the regression formula using smf.ols 
- Display the summary of the regression model using model.summary()

What is the predicted points for a driver starting in position 3 and completing 55 laps?

Hint: Use ```.dropna()''' to remove missing values from the points, grid, and laps <br>
variables.

In [49]:
# Write your answer here
# Remove non-numeric missing values from points, grid, and laps
results_cleaned = results.dropna(subset = ['points', 'grid', 'laps'])

# Run a regression of 'points ~ grid + laps'
model = smf.ols(formula = 'points ~  grid + laps', data = results_cleaned)

# Fit the model with standard errors 'cov'
model_fitted = model.fit()

# Summarize the model
print(model_fitted.summary())

# Retrieve the parameters from the model
b_list = model_fitted.params

# Print the coefficient values
print(b_list)

# Predict the points for position 3 and 55 laps
print(b_list[0] + b_list[1] * 3 + b_list[2] * 55)

                            OLS Regression Results                            
Dep. Variable:                 points   R-squared:                       0.215
Model:                            OLS   Adj. R-squared:                  0.215
Method:                 Least Squares   F-statistic:                     3530.
Date:                Wed, 19 Mar 2025   Prob (F-statistic):               0.00
Time:                        12:06:26   Log-Likelihood:                -70440.
No. Observations:               25840   AIC:                         1.409e+05
Df Residuals:                   25837   BIC:                         1.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.5841      0.054     48.267      0.0

/var/folders/tf/pxjm37j50t1f3zlbsrq8yt5h0000gn/T/ipykernel_14565/3689699895.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(b_list[0] + b_list[1] * 3 + b_list[2] * 55)


The predicted points for a driver starting in position 3 and completing 55 laps is 4.0701.